In [2]:
import torch
import torch.nn as nn
from torch.utils import data

import torchvision
import torchvision.datasets

import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score

import numpy as np

import mlflow
import mlflow.pytorch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import warnings
warnings.filterwarnings('ignore')

print("PyTorch: {}".format(torch.__version__))
print("torchvision: {}".format(torchvision.__version__))
print("sklearn: {}".format(sklearn.__version__))
print("MLFlow: {}".format(mlflow.__version__))
print("Numpy: {}".format(np.__version__))
print("Device: ", device)

PyTorch: 1.13.0.post200
torchvision: 0.14.0
sklearn: 1.1.3
MLFlow: 2.2.1
Numpy: 1.23.5
Device:  cuda:0


In [3]:
batch_size = 256
num_classes = 10
learning_rate = 0.001

In [4]:
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=None)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=None)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
x_train, y_train = train_set.data, train_set.targets
x_test, y_test = test_set.data, test_set.targets

In [7]:
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1], x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1], x_test.shape[2])

In [10]:
#function to transform to categorical, like to_categorical() in Keras
def to_one_hot(num_classes, labels):
    one_hot = torch.zeros(([labels.shape[0], num_classes]))
    for f in range(len(labels)):
        one_hot[f][labels[f]] = 1
        
    return one_hot

In [11]:
#Convert y sets into one-hot encoded format using the function
y_train = to_one_hot(num_classes, y_train)
y_test = to_one_hot(num_classes, y_test)

In [12]:
print("Shapes")
print("x_train: {}\ny_train: {}".format(x_train.shape,
y_train.shape))
print("x_test: {}\ny_test: {}".format(x_test.shape,
y_test.shape))

Shapes
x_train: torch.Size([60000, 1, 28, 28])
y_train: torch.Size([60000, 10])
x_test: torch.Size([10000, 1, 28, 28])
y_test: torch.Size([10000, 10])


In [18]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        # IN 1x28x28 OUT 16x14x14
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1, dilation=1)
        # IN 16x14x14 OUT 32x6x6
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=0, dilation=1)
        # IN 32x6x6 OUT 64x2x2
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=0, dilation=1)
        # IN 64x2x2 OUT 256
        self.flat1 = nn.Flatten()
        self.dense1 = nn.Linear(in_features=256, out_features=128)
        self.dense2 = nn.Linear(in_features=128, out_features=64)
        self.dense3 = nn.Linear(in_features=64, out_features=10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.flat1(x)
        x = self.dense1(x)
        x = nn.ReLU()(x)
        x = self.dense2(x)
        x = nn.ReLU()(x)
        x = self.dense3(x)
        x = nn.Softmax()(x)
        return x

In [19]:
model = model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [20]:
dataset = data.TensorDataset(x_train,y_train)
train_loader = data.DataLoader(dataset, batch_size=batch_size)

In [21]:
num_epochs = 5
for f in range(num_epochs):
    for batch_num, minibatch in enumerate(train_loader):
        minibatch_x, minibatch_y = minibatch[0], minibatch[1]
        
        output = model.forward(torch.Tensor(minibatch_x.float()).cuda())
        loss = criterion(output, torch.Tensor(minibatch_y.float()).cuda())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Epoch {f} Batch_Num {batch_num} Loss {loss}")

Epoch 0 Batch_Num 0 Loss 0.3757890462875366
Epoch 0 Batch_Num 1 Loss 0.3312697112560272
Epoch 0 Batch_Num 2 Loss 0.31889110803604126
Epoch 0 Batch_Num 3 Loss 0.29748207330703735
Epoch 0 Batch_Num 4 Loss 0.2898063063621521
Epoch 0 Batch_Num 5 Loss 0.26584184169769287
Epoch 0 Batch_Num 6 Loss 0.2531784176826477
Epoch 0 Batch_Num 7 Loss 0.2369346171617508
Epoch 0 Batch_Num 8 Loss 0.21736092865467072
Epoch 0 Batch_Num 9 Loss 0.1907939910888672
Epoch 0 Batch_Num 10 Loss 0.1657032072544098
Epoch 0 Batch_Num 11 Loss 0.15756025910377502
Epoch 0 Batch_Num 12 Loss 0.1422877460718155
Epoch 0 Batch_Num 13 Loss 0.14614112675189972
Epoch 0 Batch_Num 14 Loss 0.13269813358783722
Epoch 0 Batch_Num 15 Loss 0.11394134908914566
Epoch 0 Batch_Num 16 Loss 0.10910794883966446
Epoch 0 Batch_Num 17 Loss 0.09012476354837418
Epoch 0 Batch_Num 18 Loss 0.12695950269699097
Epoch 0 Batch_Num 19 Loss 0.09045200794935226
Epoch 0 Batch_Num 20 Loss 0.0812843069434166
Epoch 0 Batch_Num 21 Loss 0.07577208429574966
Epoch 0

In [25]:
mlflow.set_experiment("PyTorch_MNIST")

with mlflow.start_run():
    
    preds = model.forward(torch.Tensor(x_test.float()).cuda())
    preds = np.round(preds.detach().cpu().numpy())
    
    eval_acc = accuracy_score(y_test, preds)
    auc_score = roc_auc_score(y_test, preds)
    
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("learning_rate", learning_rate)
    
    mlflow.log_metric("eval_acc", eval_acc)
    mlflow.log_metric("auc_score", auc_score)
    
    print("eval_acc: ", eval_acc)
    print("auc_score: ", auc_score)
    
    mlflow.pytorch.log_model(model, "PyTorch_MNIST")
mlflow.end_run()

2023/03/07 10:39:09 INFO mlflow.tracking.fluent: Experiment with name 'PyTorch_MNIST' does not exist. Creating a new experiment.


eval_acc:  0.9786
auc_score:  0.9881924592706973
